In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Thyroid_Cancer"
cohort = "GSE107754"

# Input paths
in_trait_dir = "../../input/GEO/Thyroid_Cancer"
in_cohort_dir = "../../input/GEO/Thyroid_Cancer/GSE107754"

# Output paths
out_data_file = "../../output/preprocess/Thyroid_Cancer/GSE107754.csv"
out_gene_data_file = "../../output/preprocess/Thyroid_Cancer/gene_data/GSE107754.csv"
out_clinical_data_file = "../../output/preprocess/Thyroid_Cancer/clinical_data/GSE107754.csv"
json_path = "../../output/preprocess/Thyroid_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A novel genomic signature predicting FDG uptake in diverse metastatic tumors"
!Series_summary	"Purpose: Building a universal genomic signature predicting the intensity of FDG uptake in diverse metastatic tumors may allow us to understand better the biological processes underlying this phenomenon and their requirements of glucose uptake."
!Series_summary	"Methods: A balanced training set (n=71) of metastatic tumors including some of the most frequent histologies, with matched PET/CT quantification measurements and whole human genome gene expression microarrays, was used to build the signature. Selection of microarray features was carried out exclusively on the basis of their strong association with FDG uptake (as measured by SUVmean35) by means of univariate linear regression. A thorough bioinformatics study of these genes was performed and multivariable models were built by fitting several state of the art regression techniques to the training set

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the background information, we can see this dataset contains gene expression microarray data
is_gene_available = True

# 2.1 Data Availability
# For the trait variable (Thyroid Cancer), we need to check if it's in the sample characteristics
trait_row = 2  # Looking at row 2, we find 'tissue: Thyroid cancer' among other tissue types

# For age, there's no key in the sample characteristics dictionary that contains age information
age_row = None

# For gender, we can find it in row 0
gender_row = 0

# 2.2 Data Type Conversion
def convert_trait(x):
    """Convert trait data to binary values (0 for non-thyroid cancer, 1 for thyroid cancer)"""
    if isinstance(x, str):
        if 'thyroid cancer' in x.lower():
            return 1
        elif 'tissue:' in x.lower() or 'biopsy location:' in x.lower():  # Any other tissue type is not thyroid cancer
            return 0
    return None

def convert_gender(x):
    """Convert gender data to binary values (0 for female, 1 for male)"""
    if isinstance(x, str):
        if ':' in x:
            value = x.split(':', 1)[1].strip().lower()
            if value == 'female':
                return 0
            elif value == 'male':
                return 1
    return None

# Age conversion function is not needed since age data is not available
convert_age = None

# 3. Save Metadata
# Set is_trait_available based on whether trait_row is None
is_trait_available = trait_row is not None

# Use the validate_and_save_cohort_info function for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a DataFrame from the sample characteristics
    sample_characteristics = {
        0: ['gender: Male', 'gender: Female'],
        1: ['dataset: Validation set', 'dataset: Training set'],
        2: ['biopsy location: Lung', 'biopsy location: Lymph node', 'biopsy location: Primary', 'biopsy location: Liver', 
            'biopsy location: Retroperitoneal implant', 'tissue: Pancreatic cancer', 'tissue: Esophagus cancer', 
            'tissue: Breast cancer', 'tissue: Colorectal cancer', 'tissue: Ovarian cancer', 'tissue: Head&neck cancer', 
            'tissue: Lung cancer', 'tissue: Malignant Melanoma', 'tissue: Endometrial cancer', 'tissue: Cervix cancer', 
            'tissue: Soft tissue sarcoma', 'tissue: Gastric cancer', 'tissue: Unknown primary', 
            'tissue: Malignant Mesothelioma', 'tissue: Thyroid cancer', 'tissue: Testes cancer', 
            'tissue: Non Hodgkin lymphoma', 'tissue: Merkel cell carcinoma', 'tissue: Vaginal cancer', 
            'tissue: Kidney cancer', 'tissue: Cervical cancer', 'tissue: Bile duct cancer', 'tissue: Urothelial cancer'],
        3: ['suvmean35: 4.09', 'suvmean35: 8.36', 'suvmean35: 5.18', 'suvmean35: 10.74', 'suvmean35: 8.62', 
            'suvmean35: 8.02', 'suvmean35: 6.87', 'suvmean35: 4.93', 'suvmean35: 1.96', 'suvmean35: 8.83', 
            'suvmean35: 3.96', 'suvmean35: 3.38', 'suvmean35: 9.95', 'suvmean35: 5.19', 'suvmean35: 7.22', 
            'suvmean35: 5.02', 'suvmean35: 4.92', 'suvmean35: 4.99', 'suvmean35: 4.01', 'suvmean35: 2.52', 
            'suvmean35: 5.52', 'suvmean35: 8.38', 'suvmean35: 3.46', 'suvmean35: 4.07', 'suvmean35: 4.67', 
            'suvmean35: 7.09', 'suvmean35: 4.83', 'suvmean35: 6.7', 'suvmean35: 3.95', 'suvmean35: 5.03']
    }
    
    # Convert the dictionary to a DataFrame format that geo_select_clinical_features expects
    # Creating a DataFrame with sample IDs as columns and characteristics types as rows
    sample_ids = [f"Sample_{i+1}" for i in range(len(sample_characteristics[0]) if 0 in sample_characteristics else 0)]
    
    # Create empty DataFrame
    clinical_data = pd.DataFrame(index=sample_characteristics.keys(), columns=sample_ids)
    
    # Fill the DataFrame with sample characteristics
    # Since we don't have individual mappings, we'll distribute the values across samples
    # This is a simplification for demonstration purposes
    for row_idx, values in sample_characteristics.items():
        for col_idx, value in enumerate(values):
            if col_idx < len(sample_ids):
                clinical_data.loc[row_idx, sample_ids[col_idx]] = value
    
    # Extract clinical features using the provided function
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    preview = preview_df(clinical_features)
    print("Preview of clinical features:")
    print(preview)
    
    # Save the processed clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)


Preview of clinical features:
{'Sample_1': [0.0, 1.0], 'Sample_2': [0.0, 0.0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")


SOFT file: ../../input/GEO/Thyroid_Cancer/GSE107754/GSE107754_family.soft.gz
Matrix file: ../../input/GEO/Thyroid_Cancer/GSE107754/GSE107754_series_matrix.txt.gz
Found the matrix table marker in the file.


Gene data shape: (41000, 84)
First 20 gene/probe identifiers:
['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074', 'A_23_P100092', 'A_23_P100103', 'A_23_P100111', 'A_23_P100127', 'A_23_P100133', 'A_23_P100141', 'A_23_P100156', 'A_23_P100177', 'A_23_P100189', 'A_23_P100196', 'A_23_P100203', 'A_23_P100220', 'A_23_P100240', 'A_23_P10025', 'A_23_P100263']


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers
# These identifiers (A_23_P100001, etc.) appear to be Agilent microarray probe IDs rather than
# standard human gene symbols (like BRCA1, TP53, etc.)
# They need to be mapped to standard gene symbols for meaningful analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Get a more complete view to understand the annotation structure
print("\nComplete sample of a few rows:")
print(gene_annotation.iloc[:3, :4].to_string())  # Show only first few columns for readability

# Examine the gene_assignment column which contains gene information
print("\nChecking for gene information in gene_assignment column:")
sample_text = gene_annotation.loc[0, 'gene_assignment'] if 'gene_assignment' in gene_annotation.columns else "Column not found"
print(f"Sample gene_assignment text: {sample_text[:200]}...")  # Print first 200 chars

# Extract gene symbols from gene_assignment column
if 'gene_assignment' in gene_annotation.columns:
    # Create a new column with extracted gene symbols
    gene_annotation['Gene'] = gene_annotation['gene_assignment'].apply(extract_human_gene_symbols)
    
    # Display a sample of the extracted gene symbols
    print("\nSample of extracted gene symbols:")
    sample_df = gene_annotation[['ID', 'Gene']].head(10)
    print(sample_df)
    
    # Check if gene symbols were successfully extracted
    empty_genes = gene_annotation['Gene'].apply(lambda x: len(x) == 0).sum()
    print(f"\nNumber of probes with no gene symbols extracted: {empty_genes} out of {len(gene_annotation)}")
    
    # Create the mapping using our new Gene column
    mapping_data = gene_annotation[['ID', 'Gene']].copy()
    mapping_data = mapping_data[mapping_data['Gene'].apply(len) > 0]  # Filter out empty gene lists
    
    # Print sample of the mapping to confirm
    print("\nSample of probe ID to gene symbol mappings:")
    print(mapping_data.head(10))
    
    # Check the size of the mapping data
    print(f"\nTotal number of probe-to-gene mappings: {len(mapping_data)}")
else:
    print("\nError: Could not find 'gene_assignment' column in the annotation data")



Gene annotation preview:
Columns in gene annotation: ['ID', 'SPOT_ID', 'CONTROL_TYPE', 'REFSEQ', 'GB_ACC', 'GENE', 'GENE_SYMBOL', 'GENE_NAME', 'UNIGENE_ID', 'ENSEMBL_ID', 'TIGR_ID', 'ACCESSION_STRING', 'CHROMOSOMAL_LOCATION', 'CYTOBAND', 'DESCRIPTION', 'GO_ID', 'SEQUENCE']
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apopt

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in gene annotation contain probe IDs and gene symbols
# Based on the gene annotation preview, 'ID' contains probe identifiers that match the gene expression data
# 'GENE_SYMBOL' contains the gene symbols we need to map to

# 2. Get a gene mapping dataframe with the identifier and gene symbol columns
mapping_df = gene_annotation[['ID', 'GENE_SYMBOL']].copy()
mapping_df = mapping_df.rename(columns={'GENE_SYMBOL': 'Gene'})
mapping_df = mapping_df.dropna(subset=['Gene'])  # Remove rows with missing gene symbols

print(f"Created mapping dataframe with {len(mapping_df)} rows")
print("Sample of the mapping dataframe:")
print(mapping_df.head(10))

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# First, let's check our input data
print(f"\nGene data shape before mapping: {gene_data.shape}")

# Apply the mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Check the result
print(f"Gene data shape after mapping: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10].tolist())

# Save the processed gene data for future use
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Created mapping dataframe with 30936 rows
Sample of the mapping dataframe:
             ID     Gene
0  A_23_P100001  FAM174B
1  A_23_P100011    AP3S2
2  A_23_P100022     SV2B
3  A_23_P100056   RBPMS2
4  A_23_P100074     AVEN
5  A_23_P100092  ZSCAN29
6  A_23_P100103    VPS39
7  A_23_P100111      CHP
8  A_23_P100127    CASC5
9  A_23_P100133    ATMIN

Gene data shape before mapping: (41000, 84)


Gene data shape after mapping: (18488, 84)
First few gene symbols:
['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAAS', 'AACS']


Gene expression data saved to ../../output/preprocess/Thyroid_Cancer/gene_data/GSE107754.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Use the gene_data variable from the previous step (don't try to load it from file)
    print(f"Gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data
try:
    # Load the clinical data from file
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Debug the trait and gender conversion functions
    print("Sample conversion results:")
    sample_trait = list(clinical_data.iloc[2])[1]  # Get a sample from trait row (row 2)
    sample_gender = list(clinical_data.iloc[0])[1]  # Get a sample from gender row (row 0)
    print(f"Sample trait value: '{sample_trait}' -> {convert_trait(sample_trait)}")
    print(f"Sample gender value: '{sample_gender}' -> {convert_gender(sample_gender)}")
    
    # Extract clinical features with CORRECT indices from Step 2
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=2,       # Correct trait row from Step 2
        convert_trait=convert_trait,
        gender_row=0,      # Correct gender row from Step 2
        convert_gender=convert_gender,
        age_row=None,      # Age row is None as per Step 2
        convert_age=None
    )
    
    print(f"Extracted clinical data shape: {clinical_features.shape}")
    print("Preview of clinical data (first 5 samples):")
    print(clinical_features.iloc[:, :5])
    
    # Save the properly extracted clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    is_trait_available = True
except Exception as e:
    print(f"Error extracting clinical data: {e}")
    is_trait_available = False

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Debug the column names to ensure they match
        print(f"Gene data columns (first 5): {gene_data.columns[:5].tolist()}")
        print(f"Clinical data columns (first 5): {clinical_features.columns[:5].tolist()}")
        
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from thyroid cancer samples with tumor type information."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=pd.DataFrame(),  # Empty dataframe for metadata
        note="Missing essential data components for linking (trait data or gene expression data)."
    )

Gene data shape before normalization: (18488, 84)
Gene data shape after normalization: (18247, 84)


Normalized gene data saved to ../../output/preprocess/Thyroid_Cancer/gene_data/GSE107754.csv
Sample conversion results:
Sample trait value: 'biopsy location: Lung' -> 0
Sample gender value: 'gender: Male' -> 1
Extracted clinical data shape: (2, 84)
Preview of clinical data (first 5 samples):
                GSM2878070  GSM2878071  GSM2878072  GSM2878073  GSM2878074
Thyroid_Cancer         0.0         0.0         0.0         0.0         0.0
Gender                 1.0         0.0         1.0         1.0         0.0


Clinical data saved to ../../output/preprocess/Thyroid_Cancer/clinical_data/GSE107754.csv
Gene data columns (first 5): ['GSM2878070', 'GSM2878071', 'GSM2878072', 'GSM2878073', 'GSM2878074']
Clinical data columns (first 5): ['GSM2878070', 'GSM2878071', 'GSM2878072', 'GSM2878073', 'GSM2878074']
Found 84 common samples between gene and clinical data
Initial linked data shape: (84, 18249)
Preview of linked data (first 5 rows, first 5 columns):
            Thyroid_Cancer  Gender       A1BG  A1BG-AS1       A1CF
GSM2878070             0.0     1.0  16.675251  8.138944  15.194191
GSM2878071             0.0     0.0  18.589095  9.361230  15.252929
GSM2878072             0.0     1.0  17.665959  8.513994  16.779446
GSM2878073             0.0     1.0  20.260758  9.269932  19.487898
GSM2878074             0.0     0.0  17.885257  9.537095  15.190126


Linked data shape after handling missing values: (84, 18249)
For the feature 'Thyroid_Cancer', the least common label is '1.0' with 2 occurrences. This represents 2.38% of the dataset.
The distribution of the feature 'Thyroid_Cancer' in this dataset is severely biased.

For the feature 'Gender', the least common label is '1.0' with 35 occurrences. This represents 41.67% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Thyroid_Cancer/cohort_info.json
Data not usable for the trait study - not saving final linked data.
